# Lectura y preprocesamiento de datos

In [98]:
import pandas as pd
import datetime
import numpy as np
from IPython.display import clear_output

import tensorflow as tf

In [2]:
data = pd.read_excel("data.xlsb")
data["Fecha_Reporte"] = data["Fecha_Reporte"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Ocurrencia"] = data["Fecha_Ocurrencia"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Pago"] = data["Fecha_Pago"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))

In [65]:
aux = data.groupby(["Fecha_Pago"])["Importe USD"].sum().reset_index()

In [66]:
fecha_inicial = aux["Fecha_Pago"].min()
fecha_final = aux["Fecha_Pago"].max()

rango_fechas = pd.date_range(fecha_inicial, fecha_final, freq="D")

In [67]:
#Si una fecha no tiene registro, se agrega con importe 0
aux = aux.set_index("Fecha_Pago")
aux = aux.reindex(rango_fechas, fill_value = 0)
aux = aux.reset_index(names = ["Fecha_Pago", "Importe USD"])

In [69]:
X = aux[["Fecha_Pago"]]
y = aux[["Importe USD"]]

In [70]:
X["anno"] = X["Fecha_Pago"].apply(lambda x : x.year)
X["Mes"] = X["Fecha_Pago"].apply(lambda x : x.month)
X["Dia"] = X["Fecha_Pago"].apply(lambda x : x.day)
X["Dia_Semana"] = X["Fecha_Pago"].apply(lambda x : x.weekday())

In [71]:
#Ahora cada uno de estos valores los transformaremos a un encoding cíclico.
X["Dia_Semana_sin"] = X["Dia_Semana"].apply(lambda x : np.sin(2*np.pi*x/7))
X["Dia_Semana_cos"] = X["Dia_Semana"].apply(lambda x : np.cos(2*np.pi*x/7))

In [72]:
X["Mes_sin"] = X["Mes"].apply(lambda x : np.sin(2*np.pi*x/12))
X["Mes_cos"] = X["Mes"].apply(lambda x : np.cos(2*np.pi*x/12))

In [73]:
#Los días dependerán del mes, pues hay meses que tienen 30 días y otros 31, además de febrero que tiene 28 o 29.
X["Dia_sin"] = X.apply(lambda x : np.sin(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)
X["Dia_cos"] = X.apply(lambda x : np.cos(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)

In [74]:
#El año lo estandarizamos, pero comenzando desde 0.
X["anno"] = (X["anno"] - X["anno"].min()) / (X["anno"].max() - X["anno"].min())

In [75]:
columns = ["anno", "Dia_Semana_sin", "Dia_Semana_cos", "Mes_sin", "Mes_cos", "Dia_sin", "Dia_cos"]

In [79]:
#Normalizamos los valores de y
y = (y.values - y.values.mean()) / y.values.std()

In [86]:
#Creamos el X_train e y_train utilizando secuencias de un largo fijo
sequence_length = 30
X_train = []
y_train = []

for i in range(sequence_length, len(X)):
    X_train.append(X.iloc[i-sequence_length:i][columns].values)
    y_train.append(y.iloc[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [88]:
X_train.shape, y_train.shape

((5071, 30, 7), (5071, 1))

# Implementación del modelo

In [105]:
def create_model(sequence_length = 30):
    #This is a transformer model

    input_layer = tf.keras.layers.Input(shape=(sequence_length, 7))

    #Encoder
    encoder = tf.keras.layers.LSTM(128, return_sequences=True)(input_layer)
    encoder = tf.keras.layers.Dropout(0.3)(encoder)
    encoder = tf.keras.layers.LSTM(128, return_sequences=True)(encoder)
    encoder = tf.keras.layers.Dropout(0.3)(encoder)
    encoder = tf.keras.layers.LSTM(128, return_sequences=False)(encoder)
    encoder = tf.keras.layers.Dropout(0.3)(encoder)

    #Decoder
    decoder = tf.keras.layers.RepeatVector(sequence_length)(encoder)
    decoder = tf.keras.layers.LSTM(128, return_sequences=True)(decoder)
    decoder = tf.keras.layers.Dropout(0.3)(decoder)
    decoder = tf.keras.layers.LSTM(128, return_sequences=True)(decoder)
    decoder = tf.keras.layers.Dropout(0.3)(decoder)
    decoder = tf.keras.layers.LSTM(128, return_sequences=True)(decoder)
    decoder = tf.keras.layers.Dropout(0.3)(decoder)

    output_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder)

    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss="mse", optimizer="adam")

    return model

In [106]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model()
    model.fit(X_train, y_train, epochs = 1000, batch_size = 32)

    loss = model.evaluate(X_train, y_train, verbose=0)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: inf
Iteration 0 - Loss: inf
Epoch 1/1000
159/159 [==============================] - 11s 28ms/step - loss: 13442531328.0000
Epoch 2/1000
159/159 [==============================] - 4s 27ms/step - loss: 13442222080.0000
Epoch 3/1000
159/159 [==============================] - 4s 26ms/step - loss: 13441980416.0000
Epoch 4/1000
159/159 [==============================] - 4s 27ms/step - loss: 13441757184.0000
Epoch 5/1000
159/159 [==============================] - 4s 26ms/step - loss: 13441530880.0000
Epoch 6/1000
159/159 [==============================] - 4s 27ms/step - loss: 13441315840.0000
Epoch 7/1000
159/159 [==============================] - 4s 27ms/step - loss: 13441105920.0000
Epoch 8/1000
159/159 [==============================] - 4s 27ms/step - loss: 13440897024.0000
Epoch 9/1000
159/159 [==============================] - 4s 27ms/step - loss: 13440680960.0000
Epoch 10/1000
159/159 [==============================] - 4s 27ms/step - loss: 13440479232.0000
Epoch 11/1000
159/1